In [2]:
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_selection import RFE
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.decomposition import PCA
from sklearn .metrics import roc_auc_score
import statistics

f_orgnl - best 12, th=0.435: 0.6993865030674846; rf (): 0.7310657596371882; best k-nn (33): 0.6229326513213981
f_op_1 - best : ; rf: ; best k-nn (): 
f_op_2 - best : ; rf: ; best k-nn (): 
f_op_3 - best : ; rf: ; best k-nn (): 

In [3]:
# Load the data
sti_features = 'train_75_people_data.csv'
#sti_features ='features_optimized_1.csv'
#sti_features ='features_optimized_2.csv'
#sti_features ='features_optimized_3.csv'

data = pd.read_csv(sti_features)

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, precision_recall_curve

class Models_validator:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.scoring = ('accuracy', 'recall', 'roc_auc')

    def get_scores(self, clf, p1, p2):
        if clf == 'rf':
            classifier = RandomForestClassifier(n_estimators=p1, max_depth=p2, bootstrap=True, random_state=42)
            scores = cross_validate(classifier, self.x, self.y, cv=5, scoring=self.scoring, return_estimator=True)
        elif clf == 'knn':
            pca = PCA(n_components=p1)
            pca_component = pca.fit_transform(self.x)
            classifier = KNeighborsClassifier(n_neighbors=p2)
            scores = cross_validate(classifier, pca_component, self.y, cv=5, scoring=self.scoring, return_estimator=True)
        return scores

    def find_best_threshold(self, model, X_val, y_val):
        probas = model.predict_proba(X_val)[:, 1]
        precision, recall, thresholds = precision_recall_curve(y_val, probas)
        f1_scores = 2 * (precision * recall) / (precision + recall)
        best_threshold_index = np.argmax(f1_scores)
        best_threshold = thresholds[best_threshold_index]
        return best_threshold

    def maximize_score(self, clf, range1, range2, score_label="roc_auc", min_accuracy=0.0):
        p1_p2list = dict()
        p1_p2 = dict()
        p1_score = dict()
        p1_accuracy = dict()
        best_thresholds = dict()

        print(f"Training: {clf}")
        for p1 in range1:
            print(f"progress: {round(100 * p1 / range1[-1], 2)}%")
            p2_acc = dict()
            p2_score = dict()
            for p2 in range2:
                scores = self.get_scores(clf, p1, p2)
                accuracy = scores['test_accuracy'].mean()
                score = scores['test_' + score_label].mean()

                if accuracy >= min_accuracy:
                    p2_acc[p2] = accuracy
                    p2_score[p2] = score

                    # Find best threshold for the current model
                    skf = StratifiedKFold(n_splits=5)
                    thresholds = []
                    for train_index, val_index in skf.split(self.x, self.y):
                        X_train, X_val = self.x[train_index], self.x[val_index]
                        y_train, y_val = self.y[train_index], self.y[val_index]

                        if clf == 'rf':
                            classifier = RandomForestClassifier(n_estimators=p1, max_depth=p2, bootstrap=True, random_state=42)
                        elif clf == 'knn':
                            pca = PCA(n_components=p1)
                            X_train = pca.fit_transform(X_train)
                            X_val = pca.transform(X_val)
                            classifier = KNeighborsClassifier(n_neighbors=p2)

                        classifier.fit(X_train, y_train)
                        best_threshold = self.find_best_threshold(classifier, X_val, y_val)
                        thresholds.append(best_threshold)

                    best_thresholds[(p1, p2)] = np.mean(thresholds)

            p1_p2list[p1] = p2_score.values()

            if len(p2_score.keys()) > 0:
                best_score = max(p2_score.values())
                p1_score[p1] = best_score
                p2_for_best_score = self.select_key(p2_score, best_score)
                p1_p2[p1] = p2_for_best_score
                p1_accuracy[p1] = p2_acc[p2_for_best_score]

        if len(p1_score.values()) > 0:
            maxscore = max(p1_score.values())
            p1 = self.select_key(p1_score, maxscore)
            acc = p1_accuracy[p1]
            best_threshold = best_thresholds[(p1, p1_p2[p1])]

            d = {"score": maxscore, "accuracy": acc, "p1": p1, "p2": p1_p2[p1], "best_threshold": best_threshold}
            return d
        return f"No model with accuracy at least {min_accuracy}"

    def select_key(self, dictionary, value):
        for k, v in dictionary.items():
            if v == value:
                return k


In [7]:
from sklearn.model_selection import GroupShuffleSplit, StratifiedGroupKFold,GroupKFold
from sklearn.preprocessing import StandardScaler
file_features = 'train_75_people_data.csv'
feature_names = ['H_value', 'S_value', 'V_value', 'red_presence', 'brown_presence', 'blue_presence', 'pink_presence', 'white_presence','black_presence','atypical_pigment_network', 'blue-white_veil', 'asymmetry_values']

# Load the features - remember the example features are not informative
df_features = pd.read_csv(file_features)

# Make the dataset, you can select different classes (see task 0)
x = np.array(df_features[feature_names])
y =  df_features['cancer_or_not']   #now True means healthy nevus, False means something else
patient_id = df_features['person_id']

#Prepare cross-validation - images from the same patient must always stay together
num_folds = 5
group_kfold = GroupKFold(n_splits=num_folds)
group_kfold.get_n_splits(x, y, patient_id)
scaler = StandardScaler()
X_scaled_train_data = scaler.fit_transform(df_features[feature_names])


rf_cv_scores = [] # creating list of cv scores
rf_ds_list=list(range(1,16)) # creating list of depths for rf
best_rf_depth=0  # best depth found after cv

model_validator = Models_validator(X_scaled_train_data,y)

#knn_results = model_validator.maximize_score('knn',range(3,4),range(1,27),score_label="roc_auc")
#rf_results = model_validator.maximize_score('rf',range(71,72),range(1,7),score_label="roc_auc")
knn_results = model_validator.maximize_score('knn',range(1,13),range(1,27),score_label="roc_auc")
rf_results = model_validator.maximize_score('rf',range(6,98,10),range(1,7),score_label="roc_auc")

trees=rf_results["p1"]
depth=rf_results["p2"]

rf_recall=rf_results["score"]
rf_accuracy=rf_results["accuracy"]
rf_threshold=rf_results["best_threshold"]

print(f"trees: {trees}, depth: {depth}, recall: {rf_recall}, accuracy: {rf_accuracy}, threshold: {rf_threshold}")

dim=knn_results["p1"]
k=knn_results["p2"]

knn_recall=knn_results["score"]
knn_accuracy=knn_results["accuracy"]
knn_threshold=knn_results["best_threshold"]

print(f"dim: {dim}, k: {k}, recall: {knn_recall}, accuracy: {knn_accuracy}, threshold: {knn_threshold}")

Training: knn
progress: 8.33%


C:\Users\andra\AppData\Local\Temp\ipykernel_5020\91800253.py:29: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)
C:\Users\andra\AppData\Local\Temp\ipykernel_5020\91800253.py:29: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)
C:\Users\andra\AppData\Local\Temp\ipykernel_5020\91800253.py:29: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)
C:\Users\andra\AppData\Local\Temp\ipykernel_5020\91800253.py:29: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)
C:\Users\andra\AppData\Local\Temp\ipykernel_5020\91800253.py:29: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)
C:\Users\andra\AppData\Local\Temp\ipykernel_5020\91800253.py:29: RuntimeWarning: invalid value encountere

progress: 16.67%


C:\Users\andra\AppData\Local\Temp\ipykernel_5020\91800253.py:29: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)
C:\Users\andra\AppData\Local\Temp\ipykernel_5020\91800253.py:29: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)
C:\Users\andra\AppData\Local\Temp\ipykernel_5020\91800253.py:29: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)
C:\Users\andra\AppData\Local\Temp\ipykernel_5020\91800253.py:29: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)
C:\Users\andra\AppData\Local\Temp\ipykernel_5020\91800253.py:29: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)
C:\Users\andra\AppData\Local\Temp\ipykernel_5020\91800253.py:29: RuntimeWarning: invalid value encountere

progress: 25.0%
progress: 33.33%
progress: 41.67%
progress: 50.0%
progress: 58.33%


...

progress: 37.5%


C:\Users\andra\AppData\Local\Temp\ipykernel_5020\91800253.py:29: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)


progress: 47.92%


C:\Users\andra\AppData\Local\Temp\ipykernel_5020\91800253.py:29: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)


progress: 58.33%
progress: 68.75%
progress: 79.17%
progress: 89.58%
progress: 100.0%
trees: 76, depth: 6, recall: 0.7620161392807219, accuracy: 0.6893299785667691, threshold: 0.4673888833986119
dim: 3, k: 24, recall: 0.7399078663108399, accuracy: 0.6920976609822012, threshold: 0.4166666666666667
